In [59]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, FunctionTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier

## Modelagem

Para construção da máquina preditiva, é de total importância que o modelo seja alimentado com dados de qualidade. Dada a análise exploratória, devemos contruir um modelo capaz de predizer com maior acertividade do que um simples método de sempre retornar o valor de `No Failure`, pois devido ao desbalanceamento, esses simples chute terá uma alta pontuanção, a depender da métrica utilizada.

Importante ressaltar que, dada a descrição do dicionário dos dados para "machine failure", poderíamos construir uma estrutura de código baseada em condicões, sem qualquer modelo matemático que também poderia obter uma ótima performance.

Para a base de dados em questão, temos um problema de classificação. E, ainda levando em consideração a eda, inicialmente, devido a simplicidade da relação dos dados, e as poucas quantidades de features e amostras, podemos primeiramente avaliar a performance de modelos lineares, como uma regressão logística ou SVM, e posteriormente, verificar como modelos baseados em árvore se comportam, como Random Forest ou Ensembles.

### Pre-processamento dos Dados

In [40]:
df = pd.read_csv("desafio_manutencao_preditiva_treino.csv")
df.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
0,1,M14860,M,298.1,308.6,1551,42.8,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,No Failure
2,5,L47184,L,298.2,308.7,1408,40.0,9,No Failure
3,6,M14865,M,298.1,308.6,1425,41.9,11,No Failure
4,7,L47186,L,298.1,308.6,1558,42.4,14,No Failure


In [52]:
x = df.drop('failure_type', axis=1)
y = df[['failure_type']]

x.shape, y.shape

((6667, 8), (6667, 1))

#### Outliers

Com as informações obtidas das análises gráficas, vimos que as as variáveis `rotational_speed_rp` e `torque_nm` possuem alguns dados que extrapolam os valores máximos dos boxplots, contudo, como ambas as variáveis são normalmente distribuídas, podemos usar o método de z-score para tratar possíveis outliers. Onde iremos medir quão longe nossos dados estão da média, e para cada dado que ultrapassar um determinado threshold, iremos removê-lo.

In [28]:
def remove_outliers(df, column_name, threshold=3):
    mean = df[column_name].mean()
    std = df[column_name].std()
    z_scores = (df[column_name] - mean) / std
    df = df[(np.abs(z_scores) < threshold)]
    return df

#### Pipeline to encoding and scaling

A etapa de codificação será aplicada somente a variávels categórica `type`, pois as variáveis de identificação dos produtos serão dropadas posteriormente, não sendo interessantes para o problema atual, além de que, nossa variável `'type`, possui um carater ordinal implícito. Para a etapa de escalonamento das variáveis, usaremos o método min-max normalization, já que nossa base total não possui uma distribuição normal, e também serão retirados os outliers.

In [33]:
categorical_transforms = Pipeline(
    ('label_encoder', LabelEncoder())
)
numerical_transforms = Pipeline([
    ('scaler', MinMaxScaler())
])

In [50]:
categorical_columns = x.select_dtypes(include=['object']).columns.values.tolist()
numerical_columns = x.select_dtypes(exclude=['object']).columns.values.tolist()
print(f"Atributos categóricos: {categorical_columns}")
print(f"Atributos numéricos: {numerical_columns}")

Atributos categóricos: ['type']
Atributos numéricos: ['air_temperature_k', 'process_temperature_k', 'rotational_speed_rpm', 'torque_nm', 'tool_wear_min']


#### Seleção das variáveis

1 - Vamos eliminar as variáveis que não fornecem informações relevantes para o problema, como as variáveis de identificação.

In [43]:
x = x.drop(columns=['udi', 'product_id'])
x.head()

,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min
0,M,298.1,308.6,1551,42.8,0
1,L,298.2,308.7,1408,46.3,3
2,L,298.2,308.7,1408,40.0,9
3,M,298.1,308.6,1425,41.9,11
4,L,298.1,308.6,1558,42.4,14


2 - Aqui, usaremos algumas informações do dicionário de dados para produzir novos atributos para nossa base de dados.

In [44]:
x.assign(
    too_hot=lambda f: f['process_temperature_k'] - f['air_temperature_k'],
    too_power=lambda f: ((f['rotational_speed_rpm'] * 2 * np.pi) / 60) * f['torque_nm'],
    too_used=lambda f: f['tool_wear_min'] * f['torque_nm']
    )

,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,too_hot,too_power,too_used
0,M,298.1,308.6,1551,42.8,0,10.5,6951.590560,0.0
1,L,298.2,308.7,1408,46.3,3,10.5,6826.722724,138.9
2,L,298.2,308.7,1408,40.0,9,10.5,5897.816608,360.0
3,M,298.1,308.6,1425,41.9,11,10.5,6252.554779,460.9
4,L,298.1,308.6,1558,42.4,14,10.5,6917.703247,593.6
...,...,...,...,...,...,...,...,...,...
6662,L,298.8,308.3,1634,27.9,12,9.5,4774.027028,334.8
6663,M,298.8,308.4,1604,29.5,14,9.6,4955.129373,413.0
6664,H,298.9,308.4,1632,31.8,17,9.5,5434.703963,540.6
6665,H,299.0,308.7,1408,48.5,25,9.7,7151.102638,1212.5


In [53]:
# def get_difference_temperature(df):
#     df['too_hot'] = df['process_temperature_k'] - df['air_temperature_k']
#     return df

# def get_power(df):
#     df['power'] = ((df['rotational_speed_rpm'] * 2 * np.pi) / 60) * df['torque_nm']
#     return df
    
# def get_usage(df):
#     df['usage'] = df['tool_wear_min'] * df['torque_nm']
#     return df

In [46]:
# feature_engineering = ColumnTransformer([
#     ('difference_temperature', FunctionTransformer(get_difference_temperature, validate=False),
#      ['process_temperature_k', 'air_temperature_k']),
#     ('power', FunctionTransformer(get_power, validate=False), 
#      ['rotational_speed_rpm', 'torque_nm']),
#     ('usage', FunctionTransformer(get_usage, validate=False),
#      ['tool_wear_min', 'torque_nm'])
# ])

#### Label Smoothing

In [60]:
def cross_validation(modelo, x, y, oversampling=False):
    
    kfold = KFold(n_splits=10)
    
    precision_list, recall_list, f_score_list = []
    
    for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(x)):
        x_split_treino = x.iloc[idx_treino, :]
        y_split_treino = y.iloc[idx_treino, :]
        
        # oversamplig
        if oversampling:
            sm = SMOTE(random_state=24)
            x_split_treino, y_split_treino = sm.fit_resample(x_split_treino, y_split_treino)
            
        modelo.fit(x_split_treino, y_split_treino.values.flatten())
        
        x_split_validacao = x.iloc[idx_validacao, :]
        y_split_validacao = y.iloc[idx_validacao, :]
        
        # validação realizada sem o oversampling
        predicoes = modelo.predict(x_split_validacao)
        
        precision, recall, f_score, _ = precision_recall_fscore_support(y_split_validacao, predicoes)
        precision_list.append(precision)
        recall_list.append(recall)
        f_score_list.append(f_score)
        
    
    return precision_list, recall_list, f_score_list
        